In [1]:


COSMOS_ENDPOINT = 'https://azcdb-mad.documents.azure.com:443/'
COSMOS_KEY = ''

DATABASE_NAME = 'Test'
CONTAINER_NAME = 'retail'


In [3]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey

In [4]:
client = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)

In [5]:
database = client.create_database_if_not_exists(id=DATABASE_NAME)

In [6]:
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id"),
    offer_throughput=400    
)

In [7]:
documento = {
    "id":"1",
    "nombre":"Producto A",
    "descripcion":"Descripción del producto A",
    "precio":29.99,
    "categoria":"Electronica"
}

type(documento)

dict

In [8]:
container.create_item(body=documento)

{'id': '1',
 'nombre': 'Producto A',
 'descripcion': 'Descripción del producto A',
 'precio': 29.99,
 'categoria': 'Electronica',
 '_rid': '0ehXAKsOt9ABAAAAAAAAAA==',
 '_self': 'dbs/0ehXAA==/colls/0ehXAKsOt9A=/docs/0ehXAKsOt9ABAAAAAAAAAA==/',
 '_etag': '"940087e0-0000-0800-0000-67199a7f0000"',
 '_attachments': 'attachments/',
 '_ts': 1729731199}

In [13]:
query = 'SELECT * FROM c'

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

print(items)

[{'id': '1', 'nombre': 'Producto A', 'descripcion': 'Descripción del producto A', 'precio': 29.99, 'categoria': 'Electronica', '_rid': '0ehXAKsOt9ABAAAAAAAAAA==', '_self': 'dbs/0ehXAA==/colls/0ehXAKsOt9A=/docs/0ehXAKsOt9ABAAAAAAAAAA==/', '_etag': '"940087e0-0000-0800-0000-67199a7f0000"', '_attachments': 'attachments/', '_ts': 1729731199}]
